## Initialization  

In [1]:
%load_ext autoreload
%autoreload 2

import os 
import sys
sys.path.insert(0, './src')
# sys.path.insert(0, '/home/kbardool/kusanagi/AdaSparseChem/src')
print(sys.path)
import time
import argparse
import yaml
import types
import copy, pprint
from time import sleep
from datetime import datetime

import numpy  as np
import torch  
import wandb
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import DataLoader 
import scipy.sparse
from scipy.special import softmax
 
from GPUtil import showUtilization as gpu_usage
# from tqdm.notebook import trange, tqdm
from tqdm     import tqdm,trange 

from envs.sparsechem_env_dev import SparseChemEnv_Dev
# from utils.sparsechem_utils import load_sparse, load_task_weights, class_fold_counts, fold_and_transform_inputs, print_metrics_cr
from dataloaders.chembl_dataloader_dev import ClassRegrSparseDataset_v3, ClassRegrSparseDataset, InfiniteDataLoader
from utils.util import ( makedir, print_separator, create_path, print_yaml, print_yaml2, print_loss, should, 
                         fix_random_seed, read_yaml, timestring, print_heading, print_dbg, save_to_pickle, load_from_pickle,
                         print_underline, write_config_report, display_config, get_command_line_args, is_notebook) 

print(' Cuda is available  : ', torch.cuda.is_available())
print(' CUDA device count  : ', torch.cuda.device_count())
print(' CUDA current device: ', torch.cuda.current_device())
print(' GPU Processes      : \n', torch.cuda.list_gpu_processes())
print()

for i in range(torch.cuda.device_count()):
    print(f" Device : cuda:{i}")
    print('   name:       ', torch.cuda.get_device_name())
    print('   capability: ', torch.cuda.get_device_capability())
    print('   properties: ', torch.cuda.get_device_properties(i))
    ## current GPU memory usage by tensors in bytes for a given device
    print('   Allocated : ', torch.cuda.memory_allocated(i) ) 
    ## current GPU memory managed by caching allocator in bytes for a given device, in previous PyTorch versions the command was torch.cuda.memory_cached
    print('   Reserved  : ', torch.cuda.memory_reserved(i) )   
    print()

gpu_usage()                             

pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=6, linewidth=132)
# torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=None, profile=None, sci_mode=None)
pd.options.display.width = 132

os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Training.ipynb"

# sys.path.pop(0)
# print(sys.path)
# is_notebook()

['./src', '/home/kbardool/kusanagi/AdaSparseChem', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python39.zip', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/lib-dynload', '', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/IPython/extensions', '/home/kbardool/.ipython']
 Cuda is available  :  True
 CUDA device count  :  1
 CUDA current device:  0
 GPU Processes      : 
 GPU:0
no processes are running

 Device : cuda:0
   name:        NVIDIA GeForce GTX 970M
   capability:  (5, 2)
   properties:  _CudaDeviceProperties(name='NVIDIA GeForce GTX 970M', major=5, minor=2, total_memory=3071MB, multi_processor_count=10)
   Allocated :  0
   Reserved  :  0

| ID | GPU  | MEM |
-------------------
|  0 | nan% |  1% |


## Create Environment

In [150]:
from src.utils.notebook_modules import initialize, init_dataloaders, init_environment, init_wandb, \
                                       training_prep, disp_dataloader_info,disp_info_1, disp_for_excel, warmup_phase, weight_policy_training

### Parse Input Args  - Read YAML config file

In [3]:
input_args = " --config yamls/chembl_3task_train.yaml " \
             " --exp_id      14oarkpu" \
             " --exp_name    0304_1549" \
             " --exp_desc    Train with dropout 0.5" \
             " --seed_idx    0 "\
             " --batch_size  128"
# get command line arguments

In [71]:
input_args = " --config yamls/chembl_3task_train.yaml " \
             " --exp_desc    Train with dropout 0.5" \
             " --seed_idx    0 "\
             " --batch_size  128"
# get command line arguments

In [76]:
opt, ns = initialize(input_args)


  command line parms : 
------------------------
 config...................  yamls/chembl_3task_train.yaml
 exp_id...................  None
 exp_name.................  None
 folder_sfx...............  None
 exp_desc.................  Train with dropout 0.5
 seed_idx.................  0
 batch_size...............  128
 backbone_lr..............  None
 task_lr..................  None
 decay_lr_rate............  None
 decay_lr_freq............  None
 gpu_ids..................  [0]
 cpu......................  False



##################################################
################### READ YAML ####################
##################################################


 log_dir              create folder:  ../experiments/AdaSparseChem/50x6_0305_1509_plr0.01_sp0.0001_sh0.01
 result_dir           folder exists:  ../experiments/AdaSparseChem/50x6_0305_1509_plr0.01_sp0.0001_sh0.01
 checkpoint_dir       folder exists:  ../experiments/AdaSparseChem/50x6_0305_1509_plr0.01_sp0.0001_sh0.01

-----

In [5]:
# args = get_command_line_args(ns.input_args.split())
    
# print_underline(' command line parms : ', True)
# for key, val in vars(args).items():
#     print(f" {key:.<25s}  {val}")

# # ********************************************************************
# # ****************** create folders and print options ****************
# # ********************************************************************
# print_separator('READ YAML')

# opt = read_yaml(args)

# fix_random_seed(opt["random_seed"])
    
# # opt['exp_instance'] = datetime.now().strftime("%m%d_%H%M")
# # opt['exp_instance'] = '0218_1358'     
# # opt['exp_description'] = f"Retrain phase for 0218_1358"
# # folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"

# create_path(opt)    
# print()
# print_heading(f" experiment name       : {opt['exp_name']} \n"
#               f" experiment id         : {opt['exp_id']} \n"
#               f" folder_name           : {opt['exp_folder']} \n"
#               f" experiment description: {opt['exp_description']}\n"
#               f" Random seeds          : {opt['seed_list']}\n"
#               f" Random  seed used     : {opt['random_seed']} \n"
#               f" log folder            : {opt['paths']['log_dir']}\n"
#               f" checkpoint folder     : {opt['paths']['checkpoint_dir']}"
#               f" Gpu ids               : {opt['gpu_ids']}"
#               f" Seed index            : {args.seed_idx}"
#               f" policy_iter           : {opt['train']['policy_iter']}"
#               f" Data Split ratios     : {opt['dataload']['x_split_ratios']}", verbose = True)

# ns.config_filename = 'run_config_seed_%04d.txt' % (opt['random_seed'])
# write_config_report(opt, filename = ns.config_filename)    
# # display_config(opt)
# ns.best_results = {}
 

In [77]:
pp.pprint(ns)

print(ns.__dict__.keys())

namespace(args=Namespace(config='yamls/chembl_3task_train.yaml', exp_id=None, exp_name=None, folder_sfx=None, exp_desc='Train with dropout 0.5', seed_idx=0, batch_size=128, backbone_lr=None, task_lr=None, decay_lr_rate=None, decay_lr_freq=None, gpu_ids=[0], cpu=False),
          config_filename='run_config_seed_0088.txt')
dict_keys(['args', 'config_filename'])


In [78]:
# opt['exp_instance'] = '0218_1358'     
# folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"
# print()
# opt['exp_instance'] = datetime.now().strftime("%m%d_%H%M")
# opt['exp_description'] = f"No Alternating Weight/Policy - training all done with both weights and policy"
# folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"

### Setup Dataloader and Model  

In [79]:
dldrs = init_dataloaders(opt)

##################################################
############### CREATE DATALOADERS ###############
##################################################


In [80]:
print(opt['train']['weight_iter_alternate'] , opt['train']['alpha_iter_alternate'])

105 105


In [82]:
print(ns.__dict__.keys())
print(dldrs.__dict__.keys())

dict_keys(['args', 'config_filename'])
dict_keys(['trainset0', 'trainset1', 'trainset2', 'valset', 'testset', 'warmup_trn_loader', 'weight_trn_loader', 'policy_trn_loader', 'val_loader', 'test_loader'])


In [84]:
environ = init_environment(ns, opt, is_train = True, policy_learning = False, display_cfg = False)

##################################################
############# CREATE THE ENVIRONMENT #############
##################################################
 device is  cuda:0
--------------------------------------------------------
* SparseChemEnv_Dev environment successfully created
-------------------------------------------------------- 

 cuda available [0]


In [23]:
# # ********************************************************************
# # ********************Create the environment *************************
# # ********************************************************************
# # create the model and the pretrain model
# print_separator('CREATE THE ENVIRONMENT')
# environ = SparseChemEnv_Dev(log_dir          = opt['paths']['log_dir'], 
#                             checkpoint_dir   = opt['paths']['checkpoint_dir'], 
#                             exp_name         = opt['exp_name'],
#                             tasks_num_class  = opt['tasks_num_class'], 
#                             init_neg_logits  = opt['train']['init_neg_logits'], 
#                             device           = opt['gpu_ids'][0],
#                             init_temperature = opt['train']['init_temp'], 
#                             temperature_decay= opt['train']['decay_temp'], 
#                             is_train         = True,
#                             opt              = opt, 
#                             verbose          = opt['verbose'])
# environ.define_optimizer(policy_learning=False)
# environ.define_scheduler(policy_learning=False)

# cfg = environ.print_configuration()
# # print(cfg)
# write_config_report(opt, cfg, filename = config_filename, mode = 'a')

###  Weights and Biases Initialization 

In [103]:
# init_wandb(ns, opt, environment = environ)

3lmm6sdm 0305_1509 AdaSparseChem


 PROJECT NAME: AdaSparseChem
 RUN ID      : 3lmm6sdm 
 RUN NAME    : 0305_1509


In [104]:
# print(ns.__dict__.keys())
# print(ns.wandb_run.project)
# print(ns.wandb_run)
 

dict_keys(['args', 'config_filename', 'wandb_run', 'print_freq', 'eval_iters', 'stop_iter_w', 'stop_iter_a', 'flag', 'current_epoch', 'current_iter', 'best_results', 'best_metrics', 'best_value', 'best_iter', 'p_epoch', 'w_epoch', 'flag_warmup', 'num_prints', 'num_blocks', 'warm_up_epochs', 'train_total_epochs', 'curriculum_speed', 'stop_epoch_warmup'])
AdaSparseChem


In [105]:
print(f" PROJECT NAME: {ns.wandb_run.project}\n"
      f" RUN ID      : {ns.wandb_run.id} \n"
      f" RUN NAME    : {ns.wandb_run.name}") 

 PROJECT NAME: AdaSparseChem
 RUN ID      : 3lmm6sdm 
 RUN NAME    : 0305_1509


In [39]:
# print(opt['exp_id'], opt['exp_name'], opt['project_name']) # , opt['exp_instance'])
# # opt['exp_id'] = wandb.util.generate_id()

# run = wandb.init(project=opt['project_name'], entity="kbardool", resume="allow", id = opt['exp_id'], name = opt['exp_name'])
 
# print(f"PROJECT NAME: {wandb.run.project} RUN ID:  {wandb.run.id}    RUN NAME: {wandb.run.name}") 

# # # wandb.init(id='1q3dt2en' )
# # assert wandb.run is None, "Run is still running"

# wandb.config = opt.copy()
# wandb.watch(environ.networks['mtl-net'], log='all', log_freq=10)     ###  Weights and Biases Initialization 

# # wandb_run_name = opt['exp_instance']
# # wandb_run_id = wandb.util.generate_id()
# # print(wandb.run.id, wandb.run.name) 
# # print(opt['exp_uid'], opt['exp_instance'])
# # run = wandb.init(project="AdaSparseChem", entity="kbardool", resume="allow", id = opt['exp_id'], name = opt['exp_instance'])

# # wandb.config = opt.copy()
# # wandb.watch(environ.networks['mtl-net'], log='all', log_freq=10)
# environ.device

In [87]:
ns.wandb_run.finish()

### Initiate / Resume Training Prep

In [10]:
print(opt['train']['which_iter'])
a = environ.load_checkpoint('warmup_ep_40_seed_0088', path = '../experiments/AdaSparseChem/50x6_0304_1549_plr0.01_sp0.0001_sh0.01/')

warmup
=> loading snapshot from ../experiments/AdaSparseChem/50x6_0304_1549_plr0.01_sp0.0001_sh0.01/warmup_ep_40_seed_0088_model.pth.tar
   Loading to GPU cuda:0


In [11]:
opt['train']['resume'] = True
opt['train']['which_iter'] = 'warmup_ep_40_seed_0088'
if opt['train']['resume']:
    print_separator('Resume training')
    current_iter = environ.load_checkpoint(opt['train']['which_iter'])
    environ.networks['mtl-net'].reset_logits()
else:
    print_separator('Initiate Training ')

##################################################
################ Resume training #################
##################################################
=> loading snapshot from ../experiments/AdaSparseChem/50x6_0304_1549_plr0.01_sp0.0001_sh0.01/warmup_ep_40_seed_0088_model.pth.tar
   Loading to GPU cuda:0


### Training Preparation

In [106]:
print(opt['gpu_ids'])
print(opt['train']['print_freq'])
print(opt['train']['val_iters'],  'eval_iters: ', ns.eval_iters )

[0]
-1
-1 eval_iters:  33


In [107]:
training_prep(ns, opt, environ, dldrs)

 set print_freq to length of train loader: 105
 set eval_iters to length of val loader  : 33


In [108]:
# if torch.cuda.is_available():
#     print(' cuda available', gpu_ids)   
#     environ.cuda(gpu_ids)
# else:
#     print(' cuda not available')
#     environ.cpu()

# if opt['train']['print_freq'] == -1:
#     print(f" set print_freq to length of train loader: {len(warmup_trn_loader)}")
#     opt['train']['print_freq']    = len(warmup_trn_loader)

# if opt['train']['val_iters'] == -1:
#     print(f" set eval_iters to length of val loader  : {len(val_loader)}")
#     eval_iters    = len(val_loader)    
# else:
#     eval_iters    = opt['train']['val_iters']
 
# stop_iter_w = opt['train']['weight_iter_alternate']
# stop_iter_a = opt['train']['alpha_iter_alternate'] 
    
# # Fix Alpha -     
# flag           = 'update_w'
# environ.fix_alpha()
# environ.free_weights(opt['fix_BN'])

# current_epoch  = 0
# current_iter   = 0
# # current_iter_w = 0 
# # current_iter_a = 0

# best_metrics   = None
# best_value     = 0 
# best_iter      = 0

# p_epoch        = 0
# w_epoch        = 0


# flag_warmup    = True

# num_prints     = 0
# num_blocks     = sum(environ.networks['mtl-net'].layers)
# warm_up_epochs     = opt['train']['warm_up_epochs']
# train_total_epochs = opt['train']['training_epochs']
# curriculum_speed   = opt['curriculum_speed'] 

# stop_epoch_warmup  = current_epoch + warm_up_epochs

## Training Preparation

In [109]:
disp_dataloader_info(dldrs)


 trainset.y_class                                   :  [(13331, 5), (13331, 5), (13331, 5)] 
 trainset1.y_class                                  :  [(13331, 5), (13331, 5), (13331, 5)] 
 trainset2.y_class                                  :  [(13331, 5), (13331, 5), (13331, 5)] 
 valset.y_class                                     :  [(4137, 5), (4137, 5), (4137, 5)]  
 testset.y_class                                    :  [(920, 5), (920, 5), (920, 5)]  
                                 
 size of training set 0 (warm up)                   :  13331 
 size of training set 1 (network parms)             :  13331 
 size of training set 2 (policy weights)            :  13331 
 size of validation set                             :  4137 
 size of test set                                   :  920 
                               Total                :  45050 
                                 
 lenght (# batches) in training 0 (warm up)         :  105 
 lenght (# batches) in training 1 (network p

In [110]:
disp_info_1(ns, opt, environ)


 Num_blocks                : 6                                

 # batches in Weight training epoch : 105 
 # batches in Policy training epoch : 105 
 batch size                : 128 
 Total iterations          : 25000 
 Warm-up iterations        : None 
 Warm-up epochs            : 10 
 Warm-up stop              : 10 
 train_total_epochs        : 50                                 

 Print Frequency           : -1 
 Validation Frequency      : 500 
 Validation Iterations     : -1 
 eval_iters                : 33 
 which_iter                : warmup 
 train_resume              : False                                 
                                 
 
 fix BN parms              : False 
 Backbone LR               : 0.001 
 Backbone LR               : 0.001                                 

 Sharing  regularization   : 0.01 
 Sparsity regularization   : 0.0001 
 Task     regularization   : 1 
 Last Epoch                : 0  
 # of warm-up epochs to do : 10 
 stop_iter_w               

In [111]:
disp_for_excel(opt)


 folder: 50x6_0305_1509_plr0.01_sp0.0001_sh0.01 
 layers: [50, 50, 50, 50, 50, 50]                                
 
 middle dropout         : 0.5 
 last dropout           : 0.5 
 diff_sparsity_weights  : False 
 skip_layer             : 0 
 is_curriculum          : False 
 curriculum_speed       : 3                               
 
 decay_lr_rate          : 0.85 
 decay_lr_freq          : 2000 
 policy_decay_lr_rate   : 0.85 
 policy_decay_lr_freq   : 2200                               
 
 policy_lr              : 0.01 
 lambda_sparsity        : 0.0001 
 lambda_sharing         : 0.01                               
 
 lambda_tasks           : 1 
 init_temp              : 4 
 decay_temp             : 0.965 
 decay_temp_freq        : 2 
 init_method            : random 
 init_neg_logits        : None 
 hard_sampling          : False 
 Warm-up epochs         : 10 
 training epochs        : 50


## Warmup Training

In [112]:
# environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])

In [113]:
print(ns.stop_epoch_warmup, ns.current_epoch)
ns.stop_epoch_warmup = 40
print(ns.stop_epoch_warmup, ns.current_epoch)

10 0
40 0


In [114]:
print(f" Last Epoch: {ns.current_epoch}   # of warm-up epochs to do:  {ns.warm_up_epochs} - Run epochs {ns.current_epoch+1} to {ns.stop_epoch_warmup}")

 Last Epoch: 0   # of warm-up epochs to do:  10 - Run epochs 1 to 40


In [115]:
warmup_phase(ns,opt, environ, dldrs)

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
    1 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |   10.3296   2.0791e+00   1.9979e-02   12.4288 |   0.68932   0.57139   0.57369   0.57097 |   10.3409   2.0791e+00   1.9979e-02    12.4401 |  18.9 |
    2 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |   10.0835   2.0791e+00   1.9979e-02   12.1826 |   0.68340   0.62562   0.62452   0.62533 |   10.2509   2.0791e+00   1.9979e-02    12.3501 |  21.5 |
    3 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    9.6564   2.0791e+00   1.9979e-02   11.7555 |   0.67119   0.65147   0.65721   0.65119 |   10.0698   2.0791e+00   1.9979e-02    12.1689 |  20.8 |
    4 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    9.3069   2.0791e+00   1.9979e-02   11.4061 |   0.65921   0.66746   0.67416   0.66721 |    9.8827   2.0791e+00   1.9979e-02    11.9819 |  20

   40 |   7.22e-04   7.22e-04   1.00e-02  4.000e+00 |    0.8751   2.0791e+00   1.9979e-02    2.9742 |   0.85439   0.79894   0.80125   0.79884 |   12.8435   2.0791e+00   1.9979e-02    14.9426 |  21.4 |
[Final] ep:40  it:4200 -  Total Loss: 14.9426     
Task: 12.8435   Sparsity: 2.07915e+00    Sharing: 1.99785e-02 

 epch:  40   softmax       sel        softmax        sel        softmax        sel 
 -----  -----------------  ---    -----------------  ---    -----------------  --- 
   1    0.4997     0.5003   0     0.4994     0.5006   0     0.5000     0.5000   1
   2    0.4998     0.5002   0     0.5003     0.4997   1     0.5006     0.4994   1
   3    0.4993     0.5007   0     0.5007     0.4993   1     0.5004     0.4996   1
   4    0.5002     0.4998   1     0.4999     0.5001   0     0.5001     0.4999   1
   5    0.4991     0.5009   0     0.4996     0.5004   0     0.5002     0.4998   1
   6    0.4993     0.5007   0     0.5005     0.4995   1     0.4998     0.5002   0



 epch:  40   logits  

In [116]:
# line_count = 0

# while current_epoch < stop_epoch_warmup:
#     start_time = time.time()
#     current_epoch+=1
#     #-----------------------------------------
#     # Train & Update the network weights
#     #-----------------------------------------   
#     with trange(+1, stop_iter_w+1 , initial = 0 , total = stop_iter_w, position=0, file=sys.stdout,
#                 leave= False, desc=f" Warmup Epoch {current_epoch}/{stop_epoch_warmup}") as t_warmup :
#         for _ in t_warmup:
#             current_iter += 1            

#             environ.train()    
#             batch = next(warmup_trn_loader)            
#             environ.set_inputs(batch, warmup_trn_loader.dataset.input_size)
#             environ.optimize(opt['lambdas'], 
#                              is_policy=False, 
#                              flag='update_w', 
#                              verbose = False)
        
#             t_warmup.set_postfix({'curr_iter':current_iter, 
#                                   'Loss': f"{environ.losses['total']['total'].item():.4f}"})

#         trn_losses = environ.losses
#         environ.print_trn_metrics(current_epoch, current_iter, start_time, title = f"[Warmup Trn]")
#         wandb.log(environ.losses)

#         ##--------------------------------------------------------------- 
#         ## validation
#         ##--------------------------------------------------------------- 
#         val_metrics = environ.evaluate(val_loader,
#                                        is_policy       = False, 
#                                        num_train_layers= None,
#                                        eval_iters      = eval_iters, 
#                                        progress        = True,
#                                        leave           = False,
#                                        verbose         = False)

#         environ.print_val_metrics(current_epoch, current_iter, start_time, title = f"[Warmup Val]")    
#         print_metrics_cr(current_epoch,  time.time() - start_time, trn_losses, environ.val_metrics, line_count, 
#                          out=[sys.stdout, environ.log_file], to_tqdm = True) 
#         line_count += 1
#         wandb.log(environ.val_metrics)

In [119]:
# environ.save_checkpoint('warmup', current_iter, current_epoch)   
# print_loss(environ.val_metrics, f"\n[e] Warmup epoch:{current_epoch}    iter:{current_iter}")
# environ.display_trained_policy(ns.current_epoch)
# environ.log_file.flush()

In [120]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")
print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #

 Backbone Learning Rate      : 0.001
 Tasks    Learning Rate      : 0.001
 Policy   Learning Rate      : 0.01

 Sparsity regularization     : 0.0001
 Sharing  regularization     : 0.01 

 Tasks    regularization     : 1   
 Gumbel Temp                 : 4.0000         
 Gumbel Temp decay           : 2


In [18]:
# environ.opt['train']['policy_lr']       = 0.01
# opt['train']['policy_lr']       = 0.01
# environ.opt['train']['lambda_sparsity'] = 0.1
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# environ.opt['train']['decay_temp_freq'] = 2
# print(environ.optimizers['alphas'].param_groups)
# print(environ.optimizers['alphas'].param_groups[0]['initial_lr'],environ.optimizers['alphas'].param_groups[0]['lr'],)
# print(environ.optimizers['weights'].param_groups[0]['initial_lr'], environ.optimizers['weights'].param_groups[0]['lr'])
# print(environ.optimizers['weights'].param_groups[1]['initial_lr'], environ.optimizers['weights'].param_groups[1]['lr'])

## Weight & Policy Training

### Training Preparation

In [131]:
if ns.flag_warmup:
    print_heading( f"** {timestring()} \n"
                   f"** Training epoch: {ns.current_epoch} iter: {ns.current_iter}   flag: {ns.flag} \n"
                   f"** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)\n"
                   f"** Switch from Warm Up training to Alternate training Weights & Policy \n"
                   f"** Take checkpoint and block gradient flow through Policy net", verbose=True)
    environ.define_optimizer(policy_learning=True)
    environ.define_scheduler(policy_learning=True)
    ns.flag_warmup = False
    ns.flag = 'update_w'
    environ.fix_alpha()
    environ.free_weights(opt['fix_BN'])

Batches in weight epoch (stop_iter_w): 105
Batches in policy epoch (stop_iter_a): 105



In [143]:
# current_epoch = 40
print(ns.current_epoch,ns.train_total_epochs)
 

40 50
90


In [146]:
# num_train_layers = None 
# environ.opt['is_curriculum'] = True
# environ.opt['curriculum_speed'] = 4
ns.num_train_layers = None

In [148]:
# train_total_epochs = 10
# stop_epoch_training = current_epoch +train_total_epochs

In [147]:
print(f"current_epoch           : {ns.current_epoch}") 
print(f"current_iters           : {ns.current_iter}")  
print(f"train_total_epochs      : {ns.train_total_epochs}") 
print(f"stop_epoch_training     : {ns.stop_epoch_training}")
print(f"Batches in weight epoch : {ns.stop_iter_w}")
print(f"Batches in policy epoch : {ns.stop_iter_a}")
print()

current_epoch           : 40
current_iters           : 4200
train_total_epochs      : 50
stop_epoch_training     : 90
Batches in weight epoch : 105
Batches in policy epoch : 105



In [138]:
print_loss(environ.val_metrics, title = f"[e] Last epoch:{ns.current_epoch}  it:{ns.current_iter}")
# environ.display_trained_policy(ns.current_epoch)
# environ.display_trained_logits(ns.current_epoch)

[e] Last epoch:40  it:4200 -  Total Loss: 14.9426     
Task: 12.8435   Sparsity: 2.07915e+00    Sharing: 1.99785e-02 


In [154]:
print_heading(f" Last Epoch Completed : {ns.current_epoch}   # of epochs to run:  {ns.train_total_epochs} -->  epochs {ns.current_epoch+1} to {ns.stop_epoch_training}"
              f"\n policy_learning rate : {environ.opt['train']['policy_lr']} "
              f"\n lambda_sparsity      : {environ.opt['train']['lambda_sparsity']}"
              f"\n lambda_sharing       : {environ.opt['train']['lambda_sharing']}"
              f"\n curriculum training  : {opt['is_curriculum']}     cirriculum speed: {opt['curriculum_speed']}     num_training_layers : {ns.num_train_layers}", 
              verbose = True)

------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 40   # of epochs to run:  50 -->  epochs 41 to 50
 policy_learning rate : 0.01 
 lambda_sparsity      : 0.0001
 lambda_sharing       : 0.01
 curriculum training  : False     cirriculum speed: 3     num_training_layers : None
------------------------------------------------------------------------------------------------------------------------ 



### Weight/Policy Training

In [160]:
weight_policy_training(ns, opt, environ, dldrs, epochs = 10 )

------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 42   # of epochs to run:  50 -->  epochs 43 to 52 
 policy_learning rate : 0.01      
 lambda_sparsity      : 0.0001
 lambda_sharing       : 0.01 
 curriculum training  : False     cirriculum speed: 3     num_training_layers : None
------------------------------------------------------------------------------------------------------------------------ 

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
   43 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    0.8420   3.1854e+00   2.9997e-02    4.0574 |   0.86447   0.79700   0.79988   0.79690 |   12.9681   3.1974e+00   3.2340e-02    16.1978 |  23.4 |
   44 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    0.5892   3.1974e+00   3.2340e-02    3.819

In [157]:
# while current_epoch < stop_epoch_training:
#     current_epoch+=1

#     #-----------------------------------------------------------------------------------------------------------
#     # Set number of layers to train based on cirriculum_speed and p_epoch (number of epochs of policy training)
#     # e.g., When curriculum_speed == 3, num_train_layers is incremented  after every 3 policy training epochs
#     #-----------------------------------------------------------------------------------------------------------
#     num_train_layers = (p_epoch // opt['curriculum_speed']) + 1  if opt['is_curriculum'] else None
    
#     #-----------------------------------------
#     # Train & Update the network weights
#     #-----------------------------------------
#     if flag == 'update_w':
#         start_time = time.time()
#         environ.train()
        
#         with trange(+1, stop_iter_w+1 , initial = 0, total = stop_iter_w,  file=sys.stdout,
#                      position=0, ncols = 132, leave= leave, desc=f"Ep: {current_epoch} [weights]") as t_weights :
            
#             for _ in t_weights:    
#                 current_iter += 1

#                 batch = next(weight_trn_loader)
#                 environ.set_inputs(batch, weight_trn_loader.dataset.input_size)
#                 environ.optimize(opt['lambdas'], 
#                                  is_policy=opt['policy'], 
#                                  flag=flag, 
#                                  num_train_layers=num_train_layers,
#                                  hard_sampling=opt['train']['hard_sampling'],
#                                  verbose = False)

#                 t_weights.set_postfix({'it' : current_iter, 
#                                        'Lss': f"{environ.losses['losses']['total'].item():.4f}" , 
#                                        'Spr': f"{environ.losses['sparsity']['total'].item():.4e}",  
#                                        'Shr': f"{environ.losses['sharing']['total'].item():.4e}",
#                                        'lyr': f"{num_train_layers}"})    
 
#         trn_losses = environ.losses
#         environ.print_trn_metrics(current_epoch, current_iter, start_time, title = f"[Weight Trn]", to_display = False)
#         wandb.log(environ.losses)
                      
#         #--------------------------------------------------------------------
#         # validation process (here current_iter_w and stop_iter_w are equal)
#         #--------------------------------------------------------------------
#         val_metrics = environ.evaluate(val_loader,  is_policy=opt['policy'],
#                                        num_train_layers=num_train_layers, hard_sampling=opt['train']['hard_sampling'],
#                                        eval_iters = eval_iters, progress = True, leave = leave, verbose = False)  

#         environ.print_val_metrics(current_epoch, current_iter, start_time, title = f"[Weight Val]", verbose = False)
#         print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics, line_count, out=[sys.stdout, environ.log_file]) 
#         line_count +=1
#         wandb.log(environ.val_metrics)

#         #------------------------------------------------------------------------ 
#         #  Save Best Checkpoint Code (saved below and in sparsechem_env_dev.py)
#         #----------------------------------------------------------------------- 
#         # Take check point:     environ.save_checkpoint('latest_weights', current_iter)
#         #-----------------------------------------------------------------------
#         # END validation process 
#         #-----------------------------------------------------------------------
#         flag = 'update_alpha'
#         environ.fix_weights()
#         environ.free_alpha()
        
# #         environ.display_trained_policy(current_epoch,out=[sys.stdout])

#     #-----------------------------------------
#     # Policy Training  
#     #-----------------------------------------
#     if flag == 'update_alpha':
#         start_time = time.time()        
#         environ.train()
        
#         with trange( +1, stop_iter_a+1 , initial = 0, total = stop_iter_a,   file=sys.stdout,
#                      position=0, dynamic_ncols = True, leave= leave, desc=f"Ep:{current_epoch} [policy] ") as t_policy :
#             for _ in t_policy:    
#                 current_iter += 1
#                 batch = next(policy_trn_loader)

#                 environ.set_inputs(batch, policy_trn_loader.dataset.input_size)

#                 environ.optimize(opt['lambdas'], is_policy=opt['policy'],  
#                                  flag=flag, num_train_layers=num_train_layers,
#                                  hard_sampling=opt['train']['hard_sampling'], verbose = False)
                
#                 t_policy.set_postfix({'it' : current_iter,
#                                       'Lss': f"{environ.losses['losses']['total'].item():.4f}",
#                                       'Spr': f"{environ.losses['sparsity']['total'].item():.4e}",
#                                       'Shr': f"{environ.losses['sharing']['total'].item():.4e}",
#                                       'lyr': f"{num_train_layers}"})    
# #                                       ,'row_ids':f"{batch['row_id'][0]}-{batch['row_id'][-1]}"})

#         # print loss results - here current_iter_w and stop_iter_w are equal
#         trn_losses = environ.losses
#         environ.print_trn_metrics(current_epoch, current_iter, start_time, title = f"[Policy Trn]")
#         wandb.log(environ.losses)
        
#         #--------------------------------------------------------------------
#         # validation process (here current_iter_a and stop_iter_a are equal)
#         #--------------------------------------------------------------------        
#         val_metrics = environ.evaluate(val_loader, is_policy=opt['policy'],
#                                        num_train_layers=num_train_layers, hard_sampling=opt['train']['hard_sampling'],
#                                        eval_iters = eval_iters, progress = True, leave = False, verbose = False)  

#         environ.print_val_metrics(current_epoch, current_iter, start_time, title = f"[Policy Val]", verbose = False)
#         print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics, line_count, out=[sys.stdout, environ.log_file])      
#         line_count +=1
#         wandb.log(environ.val_metrics)
#         #-----------------------------------------------------------------------
#         # END validation process 
#         #-----------------------------------------------------------------------    
        
#         p_epoch += 1        
#         if should(p_epoch, opt['train']['decay_temp_freq']):
#             environ.decay_temperature()
#             print(f" decay gumbel softmax to {environ.gumbel_temperature}")
        
#         flag = 'update_w'
#         environ.fix_alpha()
#         environ.free_weights(opt['fix_BN'])
        
#     environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])
# #         environ.display_trained_logits(current_epoch)        
# #         print_loss(current_epoc, current_iter, environ.val_metrics, title = f"[Policy trn]  ep:{current_epoch}   it:{current_iter}")
    
#     #-----------------------------------------
#     # End Policy Training  
#     #----------------------------------------- 
#     if should(current_epoch, 5):
#         environ.save_checkpoint('model_latest_weights_policy', current_iter)        
#         print_loss(environ.val_metrics, title = f"\n[e] Policy training epoch:{current_epoch}  it:{current_iter}")
#         environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])
#         environ.log_file.flush()
#         line_count = 0

In [ ]:
Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
   41 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    0.8728   2.0798e+00   2.2045e-02    2.9746 |   0.79093   0.79385   0.79872   0.79374 |   11.8822   2.0798e+00   2.2045e-02    13.9840 |  21.3 |
   41 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    0.8624   3.3384e+00   2.4459e-02    4.2252 |   0.83149   0.79523   0.79952   0.79511 |   12.4564   3.3486e+00   2.5304e-02    15.8303 |  24.7 |

 epch:  41   softmax       sel        softmax        sel        softmax        sel 
 -----  -----------------  ---    -----------------  ---    -----------------  --- 
   1    0.6458     0.3542   1     0.6070     0.3930   1     0.5805     0.4195   1
   2    0.6702     0.3298   1     0.6470     0.3530   1     0.6384     0.3616   1
   3    0.7101     0.2899   1     0.6877     0.3123   1     0.7061     0.2939   1
   4    0.7079     0.2921   1     0.6889     0.3111   1     0.6832     0.3168   1
   5    0.6939     0.3061   1     0.6880     0.3120   1     0.6562     0.3438   1
   6    0.7013     0.2987   1     0.6764     0.3236   1     0.6851     0.3149   1


   42 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    0.7771   3.3486e+00   2.5304e-02    4.1511 |   0.83653   0.79496   0.79934   0.79485 |   12.5565   3.3486e+00   2.5304e-02    15.9304 |  22.3 |
   42 |   1.00e-03   1.00e-03   1.00e-02  4.000e+00 |    0.8742   4.3501e+00   3.0319e-02    5.2546 |   0.86560   0.79615   0.80054   0.79604 |   12.9552   4.3551e+00   2.6973e-02    17.3373 |  26.0 |
 decay gumbel softmax to 3.86

 epch:  42   softmax       sel        softmax        sel        softmax        sel 
 -----  -----------------  ---    -----------------  ---    -----------------  --- 
   1    0.6727     0.3273   1     0.6672     0.3328   1     0.6010     0.3990   1
   2    0.7368     0.2632   1     0.7413     0.2587   1     0.7077     0.2923   1
   3    0.8078     0.1922   1     0.8050     0.1950   1     0.7959     0.2041   1
   4    0.8129     0.1871   1     0.8185     0.1815   1     0.8087     0.1913   1
   5    0.7898     0.2102   1     0.7883     0.2117   1     0.7289     0.2711   1
   6    0.8033     0.1967   1     0.7912     0.2088   1     0.7895     0.2105   1


   43 |   1.00e-03   1.00e-03   1.00e-02  3.860e+00 |    1.0659   4.3551e+00   2.6973e-02    5.4481 |   0.85030   0.79606   0.80014   0.79596 |   12.7389   4.3551e+00   2.6973e-02    17.1210 |  22.3 |
   43 |   1.00e-03   1.00e-03   1.00e-02  3.860e+00 |    0.8035   5.0357e+00   2.5471e-02    5.8647 |   0.88083   0.79652   0.80068   0.79641 |   13.2055   5.0398e+00   2.5324e-02    18.2706 |  25.9 |

 epch:  43   softmax       sel        softmax        sel        softmax        sel 
 -----  -----------------  ---    -----------------  ---    -----------------  --- 
   1    0.6780     0.3220   1     0.6618     0.3382   1     0.6301     0.3699   1
   2    0.7836     0.2164   1     0.7349     0.2651   1     0.7357     0.2643   1
   3    0.8825     0.1175   1     0.8691     0.1309   1     0.8280     0.1720   1
   4    0.8726     0.1274   1     0.8646     0.1354   1     0.8581     0.1419   1
   5    0.8440     0.1560   1     0.8456     0.1544   1     0.7950     0.2050   1
   6    0.8380     0.1620   1     0.8524     0.1476   1     0.8421     0.1579   1


   44 |   1.00e-03   1.00e-03   1.00e-02  3.860e+00 |    0.8932   5.0398e+00   2.5324e-02    5.9583 |   0.89359   0.79644   0.80050   0.79634 |   13.3804   5.0398e+00   2.5324e-02    18.4455 |  22.4 |
   44 |   1.00e-03   1.00e-03   1.00e-02  3.860e+00 |    0.5556   5.3640e+00   3.5313e-02    5.9550 |   0.89871   0.79618   0.80029   0.79607 |   13.4941   5.3614e+00   2.5808e-02    18.8814 |  25.9 |
 decay gumbel softmax to 3.7249

 epch:  44   softmax       sel        softmax        sel        softmax        sel 
 -----  -----------------  ---    -----------------  ---    -----------------  --- 
   1    0.6587     0.3413   1     0.6316     0.3684   1     0.5749     0.4251   1
   2    0.8027     0.1973   1     0.7663     0.2337   1     0.7475     0.2525   1
   3    0.8963     0.1037   1     0.8871     0.1129   1     0.8511     0.1489   1
   4    0.9030     0.0970   1     0.9038     0.0962   1     0.8964     0.1036   1
   5    0.8662     0.1338   1     0.8678     0.1322   1     0.8120     0.1880   1
   6    0.8525     0.1475   1     0.8732     0.1268   1     0.8357     0.1643   1


In [21]:
model_label   = 'model_train_ep_%d_seed_%04d' % (current_epoch, opt['random_seed'])
metrics_label = 'metrics_train_ep_%d_seed_%04d.pickle' % (current_epoch, opt['random_seed'])
environ.save_checkpoint(model_label, current_iter, current_epoch) 
save_to_pickle(environ.val_metrics, environ.opt['paths']['checkpoint_dir'], metrics_label)
print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])
environ.display_trained_logits(current_epoch)
environ.log_file.flush()


# environ.save_checkpoint('best_weights_policy_2', current_iter, current_epoch) 
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

[Final] ep:60  it:4200 -  Total Loss: 20.7877     
Task: 14.1191   Sparsity: 6.63991e+00    Sharing: 2.87537e-02 

 epch:  60   softmax       sel        softmax        sel        softmax        sel 
 -----  -----------------  ---    -----------------  ---    -----------------  --- 
   1    0.6732     0.3268   1     0.6369     0.3631   1     0.5632     0.4368   1
   2    0.8367     0.1633   1     0.8318     0.1682   1     0.7858     0.2142   1
   3    0.9442     0.0558   1     0.9399     0.0601   1     0.8721     0.1279   1
   4    0.9570     0.0430   1     0.9632     0.0368   1     0.9441     0.0559   1
   5    0.9136     0.0864   1     0.9303     0.0697   1     0.8721     0.1279   1
   6    0.9061     0.0939   1     0.9375     0.0625   1     0.9047     0.0953   1



 epch:  60   logits        sel          logits       sel         logits        sel 
 -----  -----------------  ---    ----------------   ---    ----------------   --- 
   1    0.1796    -0.5430   1     0.1820    -0.3801   

In [88]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")


print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #

 Backbone Learning Rate      : 0.001
 Tasks    Learning Rate      : 0.001
 Policy   Learning Rate      : 0.002

 Sparsity regularization     : 0.1
 Sharing  regularization     : 0.01 

 Tasks    regularization     : 1   
 Gumbel Temp                 : 0.2764         
 Gumbel Temp decay           : 2


In [89]:
environ.opt['train']['policy_lr']       = 0.002
environ.opt['train']['lambda_sparsity'] = 0.1
environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# # environ.opt['train']['decay_temp_freq'] = 2
 

In [90]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")


print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #

print()
print( f" current_iters               : {current_iter}")  
print( f" current_epochs              : {current_epoch}") 
print( f" train_total_epochs          : {train_total_epochs}") 
print( f" stop_epoch_training         : {stop_epoch_training}")

 Backbone Learning Rate      : 0.001
 Tasks    Learning Rate      : 0.001
 Policy   Learning Rate      : 0.002

 Sparsity regularization     : 0.1
 Sharing  regularization     : 0.01 

 Tasks    regularization     : 1   
 Gumbel Temp                 : 0.2764         
 Gumbel Temp decay           : 2

 current_iters               : 32550
 current_epochs              : 160
 train_total_epochs          : 10
 stop_epoch_training         : 160


In [ ]:
# pp.pprint(environ.losses)
# pp.pprint(environ.val_metrics)

In [ ]:
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch)
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [ ]:
# print_loss(current_iter, environ.losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, trn_losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, environ.val_metrics, title = f"[e] Policy training epoch:{current_epoch}    iter:")

In [55]:
print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics, 0, out=[sys.stdout])

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
   90 |   5.22e-04   5.22e-04   6.14e-03  9.968e-01 |    2.3488   2.3753e+00   2.9361e-02    4.7535 |   0.66047   0.80706   0.81083   0.80696 |    9.8989   2.3745e+00   3.2183e-02    12.3056 |2719.3 |


In [ ]:
# environ.losses

environ.val_metrics

In [59]:
# environ.batch_data

# environ.display_parameters()

# with np.printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', precision=7, formatter={'float': lambda x: f"{x:12.5e}"}):
#     environ.print_logit_grads('gradients')

# environ_params = environ.get_task_specific_parameters()
# environ_params = environ.get_arch_parameters()
# environ_params = environ.get_backbone_parameters()
# print(environ_params)
# for param in environ_params:
#     print(param.grad.shape, '\n', param.grad)
#     print(param)

In [ ]:
environ.display_trained_logits(current_epoch)
environ.display_trained_policy(current_epoch)

In [ ]:
environ.display_test_sample_policy(current_epoch, hard_sampling = True)
environ.display_train_sample_policy(current_epoch, hard_sampling = True)

In [77]:
    environ.define_optimizer(policy_learning=True)

In [ ]:
print(environ.optimizers['alphas'])
print(environ.optimizers['weights'])

In [35]:
print('Policy  initial_lr : ', environ.optimizers['alphas'].param_groups[0]['initial_lr'], 'lr : ',environ.optimizers['alphas'].param_groups[0]['lr'])
print('Weights initial_lr : ', environ.optimizers['weights'].param_groups[0]['initial_lr'], 'lr : ',environ.optimizers['weights'].param_groups[0]['lr'])
print('Weights initial_lr : ', environ.optimizers['weights'].param_groups[1]['initial_lr'], 'lr : ',environ.optimizers['weights'].param_groups[1]['lr'])

Policy  initial_lr :  0.01 lr :  0.0072250000000000005
Weights initial_lr :  0.001 lr :  0.0007224999999999999
Weights initial_lr :  0.001 lr :  0.0007224999999999999


In [94]:
wandb.finish()

[autoreload of wandb.apis.normalize failed: Traceback (most recent call last):
  File "/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 855, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/wandb/apis/normalize.py", line 

[autoreload of wandb.sdk.interface.interface_shared failed: Traceback (most recent call last):
  File "/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 855, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/wandb/sdk/inter

Error in callback <function _WandbInit._resume_backend at 0x7f6cc229d310> (for pre_run_cell):


AttributeError: 'NoneType' object has no attribute 'info'

Exception: Attribute _telemetry_obj_dirty is not supported on Run object.

Error in callback <function _WandbInit._pause_backend at 0x7f6cc2173670> (for post_run_cell):


AttributeError: 'NoneType' object has no attribute 'info'

## Post Training Stuff

In [101]:
wandb.finish()

wandb: WARNING Upgrade to the 0.9.49 version of W&B Local to get the latest features. Learn more: http://wandb.me/local-upgrade


In [ ]:
# 
p = environ.get_current_state(0)

In [ ]:
pp.pprint(p)

### Post Warm-up Training stuff

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
environ.networks['mtl-net'].arch_parameters()

In [ ]:
p = environ.get_sample_policy(hard_sampling = False)
print(p)
p = environ.get_policy_prob()
print(p)
p = environ.get_policy_logits()
print(p)

# p = environ.get_current_policy()
# print(p)

In [ ]:
a = softmax([0.0, 1])
print(a)
sampled = np.random.choice((1, 0), p=a)
print(sampled)

In [ ]:
print(environ.optimizers['weights'])
print(environ.schedulers['weights'].get_last_lr())

In [ ]:
print('losses.keys      : ', environ.losses.keys())
print('losses[task]keys : ', environ.losses['task1'].keys())
pp.pprint(environ.losses)

In [ ]:
print( environ.val_metrics.keys())
# pp.pprint(val_metrics)
print(type(environ.val_metrics['aggregated']))
print()
print(type(environ.val_metrics['task1']['classification_agg']))
print()
pp.pprint(environ.val_metrics)

In [ ]:
# import pickle
# with open("val_metrics.pkl", mode= 'wb') as f:
#         pickle.dump(val_metrics, f)
    
# with open('val_metrics.pkl', 'rb') as f:    
#     tst_val_metrics = pickle.load(f)

In [ ]:
# print(environ.input.shape) 
# a = getattr(environ, 'task1_pred')
# yc_data = environ.batch['task1_data']
# print(yc_data.shape)
# yc_ind = environ.batch['task1_ind']
# print(yc_ind.shape)
# yc_hat_all = getattr(environ, 'task1_pred')
# print(yc_hat_all.shape)
# yc_hat  = yc_hat_all[yc_ind[0], yc_ind[1]]
# print(yc_hat_all.shape, yc_hat.shape)

# 
# environ.losses
# loss = {}
# for key in environ.losses.keys():
#     loss[key] = {}
#     for subkey, v in environ.losses[key].items():
#         print(f" key:  {key}   subkey: {subkey} ")
#         if isinstance(v, torch.Tensor):
#             loss[key][subkey] = v.data
#             print(f" Tensor  -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
#         else:
#             loss[key][subkey] = v
#             print(f" integer -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
# pp.pprint(tst_val_metrics)             

In [ ]:
# print('metrics.keys: ', environ.metrics.keys())
# print('metrics[task].keys: ', environ.metrics['task1'].keys())
# pp.pprint(environ.metrics['task1'])
# pp.pprint(environ.losses['task1']['total'])

In [ ]:
# title='Iteration'
# for t_id, _ in enumerate(environ.tasks):
#     task_key = f"task{t_id+1}"
# #     print_heading(f"{title}  {current_iter}  {task_key} : {val_metrics[task_key]['classification_agg']}", verbose = True)

#     for key, _  in val_metrics[task_key]['classification_agg'].items():
#         print('%s/%-20s'%(task_key, key), val_metrics[task_key]['classification_agg'][key], current_iter)
#         print(f"{task_key:s}/{key:20s}", val_metrics[task_key]['classification_agg'][key], current_iter)
#         print()
#             # print_current_errors(os.path.join(self.log_dir, 'loss.txt'), current_iter,key, loss[key], time.time() - start_time)

In [ ]:
# environ.print_loss(current_iter, start_time, metrics = val_metrics['loss'], verbose=True)
# print(opt['lambdas'])
# p = (opt['lambdas'][0] * environ.losses['tasks']['task1'])
# print(p)

# environ.print_val_metrics(current_iter, start_time, val_metrics , title='validation', verbose=True)    

In [ ]:
# print(current_iter)
# print_metrics_cr(current_iter, t1 - t0, None, val_metrics , True)
# environ.print_val_metrics(current_iter, start_time, val_metrics, title='validation', verbose = True)

In [ ]:
print(f" val_metric keys               : {val_metrics.keys()}")
print(f" loss keys                     : {val_metrics['loss'].keys()}")
print(f" task1 keys                    : {val_metrics['task1'].keys()}")
print(f" task1 classification keys     : {val_metrics['task1']['classification'].keys()}")
print(f" task1 classification_agg keys : {val_metrics['task1']['classification_agg'].keys()}")
print()
print(f" task1                       : {val_metrics['task1']['classification_agg']['loss']:5f}")
print(f" task2                       : {val_metrics['task2']['classification_agg']['loss']:5f}")
print(f" task3                       : {val_metrics['task3']['classification_agg']['loss']:5f}")
print(f" loss                        : {val_metrics['loss']['total']:5f}")
print(f" train_time                  : {val_metrics['train_time']:2f}")
print(f" epoch                       : {val_metrics['epoch']}")


### Post Weight + Policy Training Stuff 

In [ ]:
environ.networks['mtl-net'].backbone.layer_config

In [ ]:
num_blocks = 6
num_policy_layers = 6
gt =  torch.ones((num_blocks)).long()
gt0 =  torch.zeros((num_blocks)).long()
print(gt)
print(gt0)

loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
print(loss_weights)

In [ ]:
if environ.opt['diff_sparsity_weights'] and not environ.opt['is_sharing']:
    print(' cond 1')
    ## Assign higher weights to higher layers 
    loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
    print(f"{task_key} sparsity error:  {2 * (loss_weights[-num_blocks:] * environ.cross_entropy2(logits[-num_blocks:], gt)).mean()})")
    print_dbg(f" loss_weights :  {loss_weights}", verbose = True)
    print_dbg(f" cross_entropy:  {environ.cross_entropy2(logits[-num_blocks:], gt)}  ", verbose = True)
    print_dbg(f" loss[sparsity][{task_key}]: {self.losses['sparsity'][task_key] } ", verbose = True)

else:
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between \n Logits   : \n{logits[-num_blocks:]} \n and gt: \n{gt} \n", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-num_blocks:], gt)}")
    
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt[-1:])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt0[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt0[-1:])} \n")
    
    print('\n cond 3')    
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt[0:1])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt0[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt0[0:1])} \n")
        
        

In [ ]:
# flag = 'update_w'
# environ.fix_alpha
# environ.free_w(opt['fix_BN'])

flag = 'update_alpha'
environ.fix_weights()
environ.free_alpha()

In [ ]:
environ.networks['mtl-net'].num_layers

In [ ]:
print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

train_total_epochs += 5

print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

In [ ]:
# print_metrics_cr(current_epoch, time.time() - t0, None, environ.val_metrics , num_prints)      

# num_prints += 1
# t0 = time.time()

# # Take check point
# environ.save_checkpoint('latest', current_iter)
# environ.train()
# #-------------------------------------------------------
# # END validation process
# #-------------------------------------------------------       
# flag = 'update_alpha'
# environ.fix_w()
# environ.free_alpha()

In [ ]:
# dilation = 2
# kernel_size = np.asarray((3, 3))
# upsampled_kernel_size = (kernel_size - 1) * (dilation - 1) + kernel_size
# print(upsampled_kernel_size)

In [ ]:
# environ.optimizers['weights'].param_groups[0]
# for param_group in optimizer.param_groups:
#     return param_group['lr']

In [ ]:
environ.schedulers['weights'].get_last_lr()

In [ ]:
current_state = {}
for k, v in environ.optimizers.items():
    print(f'state dict for {k} = {v}')
    current_state[k] = v.state_dict()
pp.pprint(current_state)

In [ ]:
current_state = {}
for k, v in environ.schedulers.items():
    print(f'state dict for {k} = {v}')
    print(v.state_dict())

### Losses and Metrics

In [ ]:
trn_losses = environ.losses

In [ ]:
print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# pp.pprint(environ.losses)
pp.pprint(trn_losses)

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
# environ.opt['train']['Lambda_sharing'] = 0.5
# opt['train']['Lambda_sharing'] = 0.5

# environ.opt['train']['policy_lr'] = 0.001
# opt['train']['policy_lr'] = 0.001

In [ ]:
environ.losses.keys()
pp.pprint(environ.losses)

In [ ]:
tmp = environ.get_loss_dict()
print(tmp.keys())
pp.pprint(tmp)

In [ ]:
print(opt['diff_sparsity_weights'])
print(opt['is_sharing'])
print(opt['diff_sparsity_weights'] and not opt['is_sharing'])
print(environ.opt['train']['Lambda_sharing'])
print(opt['train']['Lambda_sharing'])
print(environ.opt['train']['Lambda_sparsity'])
print(opt['train']['Lambda_sparsity'])
print(environ.opt['train']['policy_lr'])
print(opt['train']['policy_lr'])

### Policy / Logit stuff

In [ ]:
from scipy.special          import softmax

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=8,linewidth=132)

#### `get_task_logits(n)` Get logits for task group n

In [ ]:
task_logits = environ.get_task_logits(1)
print(task_logits)

#### `get_arch_parameters()`: Get last used logits from network

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

#### `get_policy_logits()`:  Get Policy Logits - returns same as `get_arch_parameters()`

In [ ]:
logs = environ.get_policy_logits()
for i in logs:
    print(i, '\n')
# probs = softmax(logs, axis= -1)
# for i in probs:
#     print(i, '\n')

#### `get_policy_prob()` : Gets the softmax of the logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
for i in policy_softmaxs:
    print(i, '\n')

#### `get_sample_policy( hard_sampling = False)` : Calls test_sample_policy of network with random choices based on softmax of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
policies,logits = environ.get_sample_policy(hard_sampling = False)

for l, p, s in zip(logits, policies, policy_softmaxs) :
    for  l_row, p_row, s_row in zip(l, p, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### `get_sample_policy( hard_sampling = True)` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
hard_policies, logits = environ.get_sample_policy(hard_sampling = True)

for p,l,s in zip(hard_policies, logits, policy_softmaxs) :
    for  p_row, l_row, s_row in zip(p, l, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### Print

In [ ]:
print(f" Layer    task 1      task 2      task 3")
print(f" -----    ------      ------      ------")
for idx, (l1, l2, l3) in enumerate(zip(hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1}       {l2}       {l3}")
    

    print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
def display_trained_policy(iter):

    policy_softmaxs = environ.get_policy_prob()
    policy_argmaxs = 1-np.argmax(policy_softmaxs, axis = -1)
    print(f"  Trained polcies at iteration: {iter} ")
    print(f"                   task 1                           task 2                         task 3        ")
    print(f" Layer       softmax        select          softmax        select          softmax        select   ")
    print(f" -----    ---------------   ------       ---------------   ------       ---------------   ------   ")
    for idx, (l1,l2,l3,  p1,p2,p3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2], policy_argmaxs[0], policy_argmaxs[1], policy_argmaxs[2]),1):
        print(f"   {idx}      {l1[0]:.4f}   {l1[1]:.4f}   {p1:4d}    {l2[0]:11.4f}   {l2[1]:.4f}   {p2:4d}    {l3[0]:11.4f}   {l3[1]:.4f}   {p3:4d}")

    print()
# print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
display_trained_policy(5)

In [ ]:
print(f"                        POLICIES (SOFTMAX)                                       task 3          ")
print(f" Layer    task1              task2            task3 softmax         softmax         argmax         softmax         argmax   ")
print(f" -----    -------------     -------------     -------------   ------   ")
for idx, (l1,l2,l3, h1,h2,h3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2],hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1[0]:.4f} {l1[1]:.4f}     {l2[0]:.4f} {l2[1]:.4f}     {l3[0]:.4f} {l3[1]:.4f}    {h3}")
    
print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
# print(policy_softmaxs[2], np.argmax(1-policy_softmaxs[2], axis = -1))
print(policy_softmaxs, np.argmax(policy_softmaxs, axis = -1))

#### `get_current_logits()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
logits  = (environ.get_current_logits())
for i in logits:
    print(i ,'\n')

#### `get_current_policy()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
pols  = (environ.get_current_policy())

for i in pols:
    print(i ,'\n')

#### `gumbel_softmax()`  

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', floatmode = 'maxprec_equal')
torch.set_printoptions(precision=8,linewidth=132)

In [ ]:
print(environ.temp)
# tau = environ.temp
tau = 1
for i in range(3): 
    logits_tensor = torch.tensor(logits[0])
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=tau, hard=False).cpu().numpy() 

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=tau, hard=True).cpu().numpy()
    
    for l, gs, gh in zip(lgts, gumbel_soft, gumbel_hard):
        print(f"   {l}   \t {gs}            \t {gh}")
#     print(lgts)
#     print(gumbel_soft)
#     print(gumbel_hard)
    print()

In [ ]:
for lgts in logits:
    logits_tensor = torch.tensor(lgts)
    print(lgts)
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=1, hard=False)
    print(gumbel_soft)

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=1, hard=True)
    print(gumbel_hard)
    print()

In [ ]:
smax = scipy.special.softmax(logs, axis =1)
# smax = np.array( 
# [[0.46973792, 0.530262  ],
#  [0.45025694, 0.549743  ],
#  [0.4443086 , 0.5556915 ],
#  [0.4138397 , 0.58616036],
#  [0.4140113 , 0.5859887 ],
#  [0.42114905, 0.57885087]])

print(smax.shape)
print(smax)
print(smax[0])
print(smax[0].sum())
print(np.random.choice((1,0), p =smax[0]))

In [ ]:
logs = np.array(
[[0.33064184, 0.42053092],
 [0.3532089 , 0.52056104],
 [0.3888512 , 0.5680909 ],
 [0.42039296, 0.694217  ],
 [0.4519742 , 0.73311865],
 [0.48401102, 0.7522658 ]],
)